# 因子分析

**因子分析**（factor analysis）はデータの分散を分解することによって何らかの共通性を抽出する手法である。

心理学などの分野において、「リーダシップ」や「社交性」のような直接観測することができない**構成概念**（construct）を抽出するために用いられる。



## 因子分析モデル

### 1因子モデル

$p$次元の観測値が$n$個得られているとし、$i$番目の$j$次元のデータを$x_{ij}$と表すことにする。$x_{ij}$を標準化して平均0、分散1にしたデータを$z_{ij}$と表すことにする。

1因子モデルは標準化データを次のように分解するモデルである。

$$
z_{ij} = a_j f_i + d_j u_{ij}
$$

ここで

- $f_i$は観測対象$i$の**共通因子**（common factor）で、構成概念を表す
- $u_{ij}$は**独自因子**（unique factor）と呼ばれ、構成概念で説明しきれない要因を表す
- $a_j$は観測変数$j$の**因子負荷**（factor loading）で、観測変数$j$が共通因子から受ける影響の程度を表す
- $d_j$は観測変数$j$の**独自係数**と呼ばれ、観測変数$j$が独自因子から受ける影響の程度を表す


### $m$因子モデル

$m$個の因子を扱うように拡張すると


$$
\begin{align}
z_{ij}
&= a_{j1} f_{i1} + \cdots + a_{jm} f_{im} + d_j u_{ij}\\
&= \sum^m_{l=1} a_{jl} f_{il} + d_j u_{ij}\\
\end{align}
$$

と表すことができる。

In [28]:
import pandas as pd

# テストの点数データ
scores = pd.DataFrame([
    {"英語": 98, "国語": 95, "数学": 75},
    {"英語": 90, "国語": 75, "数学": 70},
    {"英語": 60, "国語": 57, "数学": 80},
    {"英語": 50, "国語": 70, "数学": 60},
    {"英語": 30, "国語": 50, "数学": 40},
])
Z = (scores - scores.mean()) / scores.std()

from statsmodels.multivariate.factor import Factor

fa = Factor(Z, n_factor=1, method="ml").fit()
fa.summary()

/usr/local/lib/python3.9/site-packages/statsmodels/multivariate/factor.py:419: UserWarning: Fitting did not converge
  warnings.warn("Fitting did not converge")


<class 'statsmodels.iolib.summary2.Summary'>
"""
Factor analysis results
=======================
                       
-----------------------
    Communality        
------------------------
    英語      国語      数学  
------------------------
  0.9977  0.7723  0.5347
-----------------------
                       
-----------------------
 Pre-rotated loadings  
-----------------------
               factor 0
-----------------------
英語               0.9995
国語               0.8719
数学               0.7180
-----------------------
                       
=======================

"""

In [31]:
fa.uniqueness

array([0.00228581, 0.22772564, 0.46534024])

In [36]:
# 共通因子スコア
fa.factor_scoring(Z)

array([[ 1.15171807],
       [ 0.85998399],
       [-0.19932969],
       [-0.54814633],
       [-1.26422604]])

## 主成分分析との違い

主成分分析も分散を分解して共通した成分を取り出す手法であり、因子分析と似ているが、誤差を分離するかどうかが異なる。

因子分析は観測変数を共通した部分と誤差とに分解した上で共通因子を抽出するのに対し、主成分分析は誤差の分離を行わずに共通した部分を取り出そうとする。


## 参考

- Hofacker, C. (2004). Exploratory Factor Analysis, Mathematical Marketing. http://www.openaccesstexts.org/pdf/Quant_Chapter_11_efa.pdf
- J Bai, K Li (2012). Statistical analysis of factor models of high dimension. Annals of Statistics. https://arxiv.org/pdf/1205.6617.pdf